<img src="images/logo.png" style="width: 100px;"/>

# Assignment 1

### Entscheidungsbäume, K-Fold-Cross-Validation und ILP

_Abgabefrist: **23.11.2025**_

---

#### Informationen zur Abgabe

Laden Sie Ihre Lösung über den VC-Kurs hoch. Bitte laden Sie pro Gruppe **ein Zip-Archiv** hoch. Dieses muss enthalten:

- Ihre Lösung als **Notebook** (eine `.inpynb` Datei) und/oder ggf. Python Dateien (`.py`).
- Einen Ordner **images** mit allen Ihren Bildern (halten Sie die Größe der Bilder relativ klein)

Ihre Zip-Datei sollte nach folgendem Schema benannt sein:

```
assignment_<assignment number>_solution_<group number>.zip
```

In diesem Assignment können Sie insgesamt **62** Punkte erreichen. Aus diesen Punkten berechnen sich **2,5 Bonuspunkte** für die Klausur wiefolgt:

| **Punkte im Assignment** | **Bonuspunkte für die Klausur** |
| :-: | :-: |
| 59 | 2.5 |
| 50 | 2.0 |
| 41 | 1.5 |
| 32 | 1.0 |
| 23 | 0.5 |

<div class='alert alert-block alert-danger'>

##### **Wichtige Hinweise**

1. **Dieses Assignment wird benotet. Sie können Bonuspunkte für die Prüfung erhalten.**
2. **Wenn es für uns offensichtlich ist, dass eine Aufgabe von einer anderen Quelle kopiert wurde und keine Eigenleistung erbracht wurde, vergeben wir keine Bonuspunkte. Formulieren Sie alle Antworten in eigenen Worten!**
3. **Wenn LLMs (wie z.B. ChatGPT oder CoPilot) verwendet wurden, um Ihre Abgabe zu erstellen, geben Sie dies bitte an den jeweiligen Stellen an. Beachten Sie zudem die [AI-Policy](https://cogsys.uni-bamberg.de/teaching/ki-richtlinie.html).**

---

Für Aufgabe 3 muss auf eurem System auch [SWI Prolog](https://www.swi-prolog.org) installiert sein, sonst kommt es beim installieren der Abhängigkeiten zu einem Fehler!

Du kannst _SWI Prolog_ auch über die gängigen Package-Manager installieren.

- **Windows (chocolatey):** `choco install swi-prolog`
- **macOS (Homebrew):** `brew install swi-prolog`

**Wichtig:** Wenn du _SWI Prolog_ manuell installierst, darfst du nicht vergessen es auch in deiner `PATH` Variable einzutragen.

In [ ]:
# Installiert die benötigten Pakete mit dem akutell ausgewählten Python-Interpreter
%pip install -U -r requirements.txt

### 1 | ID3 Implementierung

_Für insgesamt 26 Punkte_

Im Herbst färben sich die Wälder golden und vielerorts beginnt die Pilzsaison. Zwischen den bunten Blättern finden sich zahlreiche Pilzarten. Manche davon sind essbar, andere hochgiftig. Ein automatisiertes Verfahren zur Bestimmung der Genießbarkeit kann helfen, gefährliche Verwechslungen zu vermeiden. Hierfür können Entscheidungsbäume (Decision Trees) verwendet werden.

In dieser Aufgabe soll ein Entscheidungsbaum mithilfe des **ID3-Algorithmus** implementiert werden, um auf Grundlage von Pilzmerkmalen zu entscheiden, ob ein Pilz _genießbar_ oder _giftig_ ist. Ziel ist die Implementierung des **ID3-Algorithmus** zur Klassifikation von Pilzen auf Basis des Datensatzes `mushrooms.csv`. Der Algorithmus soll auf gegebenen Merkmalen einen Entscheidungsbaum lernen, der die Zielvariable `dangerous` möglichst genau vorhersagt.

**_Import von Bibliotheken._** In der folgenden Zelle werden einige wichtige Bibliotheken importiert. Diese sind für die fortlaufende Implementierung notwendig. Sie sollen hier kurz erläutert werden:
- `random` und `numpy`. Mit diesen beiden Bibliotheken werden _seeds_ gesetzt, um die Ergebnisse von Zufallsgeneratoren konsistent zu halten.
- `pandas`. Aus pandas verwenden wir DataFrames, in denen die Datensätze liegen.
- `typing.Any`. Wird benötigt für Type-Hints in den Spezifikationen der Methoden.
- `sklearn.base.ClassifierMixin`. Stellt gemeinsame Funktionen für Klassifikationsmodelle bereit.
- `sklearn.model_selection.train_test_split`. Teilt Datensätze in Trainings- und Testmengen auf.
- `sklearn.metrics.accuracy_score`. Berechnet den Anteil korrekt klassifizierter Beispiele.

**_An der nächsten Code Zelle muss nichts verändert werden._**

In [ ]:
import random

import numpy as np
import pandas as pd

from typing import Any
from numpy.typing import ArrayLike

from sklearn.base import ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

random.seed(2025)
np.random.seed(2025)

**_Import von Datensätzen._** In der folgenden Zelle wird der verwendete Beispieldatensatz geladen. Dieser liegt in `mushrooms.csv` und umfasst sechs kategorielle Entscheidungs- sowie ein binäres Zielattribut:

- `habitat` (`forest`, `field`, `underwater`)
- `hat_color` (`red`, `white`, `brown`)
- `size` (`small`, `medium`, `large`)
- `lamellae` (`wide`, `narrow`, `none`)
- `hat_form` (`steep`, `flat`)
- `stem` (`rough`, `smooth`)
- **Target:** `dangerous` (`True`, `False`)

Nachdem die `.csv` in einen `pd.DataFrame` geladen wurde, wird dieser in `X` (nur Entscheidungsattribute) und `y` (nur Zielattribut) geteilt. `X` und `y` werden weiterhin mit der `train_test_split()` Funktion in eine Trainingsmenge von $80\%$ und eine Testmenge von $20\%$ geteilt. **Wichtig:** mit `random_state=2025` wird sichergestellt, dass die Splits bei jeder Ausführung immer gleich sind. Scikit-Learn verwendet einen Seed, der nicht von `numpy` oder `random` gesetzt wird.

**_An der nächsten Code Zelle muss nichts verändert werden._**

In [ ]:
mushrooms = pd.read_csv('mushrooms.csv')

X = mushrooms.drop('dangerous', axis=1)
y = mushrooms['dangerous']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2025)

#### **(01.1.1)** Berechnung von Entropy und Information Gain

Für den Aufbau des Entscheidungsbaums werden zwei grundlegende Funktionen benötigt:

1. `entropy(y)`: Berechnet den Grad der Unsicherheit in der Zielverteilung
2. `information_gain(X, y, attribute)`: Misst, wie stark ein bestimmtes Attribut die Unsicherheit reduziert und somit für die Entscheidung im Baum relevant ist.

_Diese Funktionen können im nachfolgenden Code-Block implementiert werden. Gerne könnt ihr dafür aber auch eine separate Python-Datei verwenden._

In [ ]:
def entropy(y: ArrayLike) -> float:
    """
    Calculate the entropy for the example subset `y`. Note that this method uses `y`, because entropy is always calculated with respect to the target attribute, so this method only needs the target attribute.

    #### Parameters
    - `y (ArrayLike)`: Subset of the target attribute for which to calculate entropy.

    #### Returns
    - `_ (float)`: Entropy calculated for `y`.
    """
    # TODO: Compute the entropy of the example subset
    pass

In [ ]:
def information_gain(X: ArrayLike, y: ArrayLike, attribute: str) -> float:
    """
    Calculate the information gain for the attribute `attribute` given the target attribute `y`.

    #### Parameters
    - `X (ArrayLike`: The full dataset as array like structure (at this level of recursion).
    - `y (ArrayLike)`: The corresponding target attribute.
    - `attribute (str)`: The attribute for which to calculate the information gain.

    #### Returns
    - `_ (float)`: Information gain calculated for `attribute`.
    """
    # TODO: Compute the information gain
    pass

#### **(01.1.2)** Implementierung des ID3-Algorithmus

Im folgenden sollt ihr nun eine Klasse `DecisionTree` implementieren, welche den **ID3-Algorithmus** implementiert. Der Algorithmus soll rekursiv den Attributwert mit dem höchsten Information Gain auswählen und so schrittweise einen Entscheidungsbaum aufbauen. Die Baumstruktur kann frei gewählt werden (z.B. als verschachteltes Dictionary oder über eingene Klassen). Der trainierte Baum soll anschließend in Verbindung mit dem zuvor diskutierten `mushrooms.csv` Datensatz genutzt werden, um Vorhersagen über die _Genießbarkeit_ von Pilzen zu treffen.

Die Klasse `DecisionTree` soll dabei von `ClassifierMixin` erben. Dabei sind die Methoden `fit` und `predict` notwendig. Dadurch kann euer Entscheidungsbaum später problemlos in Pipelines, die auf dem `sklearn`-Framework basieren, verwendet werden.

_Es ist nicht notwendig die Implementierung in diesem Jupyter Notebook durchzuführen. Alternativ könnt ihr auch separate Python Dateien verwenden._

In [ ]:
### ID3 Implementation ###

class DecisionTree(ClassifierMixin):
    """
    Implements a tree data structure and both decision tree training using ID3 and decision tree inference. Also includes methods for calculating entropy and information gain.
    """

    def __init__(self, default_class: Any = None):
        """
        Decision Tree constructor
        """
        
        raise NotImplementedError


    def fit(self, X: ArrayLike, y: ArrayLike):
        """
        Fit the decision tree to the dataset `X` with target attribute `y`.

        #### Parameters
        - `X (v)`: The full dataset.
        - `y (ArrayLike)`: The corresponding target attribute.
        """
        
        raise NotImplementedError


    def id3(self, X: ArrayLike, y: ArrayLike) -> dict[dict] | Any:
        """
        Recursively build the decision tree using the ID3 algorithm.

        #### Parameters
        - `X (ArrayLike)`: The full dataset (at this level of recursion).
        - `y (ArrayLike)`: The corresponding target attribute.

        #### Returns
        - `_ (dict[dict] | Any)`: The resulting Decision Tree as a nested dictionary. Each node is either a nested dictionary (internal node) or a leaf node with the predicted value.
        """
        
        raise NotImplementedError


    def predict(self, X: ArrayLike) -> ArrayLike:
        """
        Predict the target attribute for the dataset `X`.

        #### Parameters
        - `X (ArrayLike)`: The dataset for which to predict the target attribute.

        #### Returns
        - `_ (ArrayLike)`: The predicted target attributes.
        """

        raise NotImplementedError


#### **(01.1.3)** Training

Nach erfolgreicher Implementierung soll der Entscheidungsbaum auf den `mushrooms.csv` Datensatz angewandt werden. Das bedeutet, dass in den folgenden Code Zellen der `DecisionTree` auf den `mushrooms.csv` Datensatz angewendet werden soll. Dabei sind die folgenden Schritte notwendig:

1. **Training des Entscheidungsbaums** mit den vorbereiteten Trainingsdaten
2. **Vorhersage** der Klassenzugehörigkeit auf den Testdaten

In [ ]:
# Train Decision Tree

# TODO

### 2 | ID3 Evaluation mit k-Fold-Cross-Validation

_Für insgesamt 20 Punkte_

Nachdem ihr im vorherigen Abschnitt den **ID3-Algorithmus** umgesetzt habt, soll der entwickelte Entscheidungsbaum (`DecisionTree`) nun bewertet werden. Ziel ist es, die Modellgüte des Baums zu bestimmen und zu überprüfen, wie stabil die erzielten Ergebnisse über verschiedene Datenaufteilungen hinweg ausfallen.

#### **(01.2.1)** k-Fold-Corss-Validation

Hierzu soll eine **k-Fold-Corss-Validation** verwendet werden. Diese Methode teilt den Datensatz in mehrere Teilmengen (Folds) auf und ermöglicht so, den Entscheidungsbaum mehrfach mit unterschiedlichen Trainings- und Testdaten zu evaluieren. Durch das wiederholte Trainieren und Testen entsteht ein umfassenderes Bild der Modellleistung, als es mit einer einmaligen Aufteilung möglich wäre.

**Dabei ist die Verwendung von `KFold` oder ähnlichen Hilfsklassen aus scikit-lear ausgeschlossen. Die Aufteilung der Daten in Folds soll eigenständig erfolgen.**

Für diese Aufgabe sollen zwei Funktionen entwickelt werden, die gemeinsam den Evaluationsprozess automatisieren:

1. `evaluate_fold`. Diese Funktion übernimmt die Bewertung eines einzelnen Folds. Sie trainiert den vorhandenen Entscheidungsbaum (`DecisionTree`) auf den Trainingsdaten und prüft anschließend dessen Accuracy auf den Testdaten. Sollte die eigene Implementierung der `DecisionTree` Klasse nicht lauffähig sein, kann auf den `DecisionTreeClassifier` aus `scikit-learn` zurückgegriffen werden, um den Evaluationsprozess dennoch vollständig durchführen zu können. In diesem Fall müssen kategoriale Eingabewerte vorab kodiert werden.
2. `run_kfold_evaluation`. Diese Funktion steuert den gesammten Ablauf der Cross-Validation. Sie erzeugt die $k$-Folds ($k=5$) des Datensatzes. Dabei sollen die Daten zunächst zufällig durchmischt werden, um eine faire Verteilung der Klassen zu gewährleisten. Anschließend werden die Indexgruppen der fünf Folds gebildet, wobei jeweils ein Fold als Test- und die übrigen als Trainingsdaten dienen. Für jeden Fold wird die zuvor definierte `evaluate_fold` Funktion aufgerufen. Alle Einzelergebnisse sollen in einer Tabelle gesammelt werden. Die resultierende Tabelle soll Angaben zu Fold-Nummer, Trainings- und Testgrößen, verwendeter Methoden und Accuracy enthalten.

In [ ]:
def evaluate_fold(
        X_train: ArrayLike,
        X_test: ArrayLike,
        y_train: ArrayLike,
        y_test: ArrayLike,
        fold: int
) -> dict:
    """
    Evaluates a single fold during a cross-validation process. The function trains the decision tree on the given training data, makes
    predictions on the test data, and calculates the accuracy of these predictions.

    #### Parameters
    - `X_train (ArrayLike)`: Training feature dataset.
    - `X_test (ArrayLike)`: Test feature dataset.
    - `y_train (ArrayLike)`: Training target labels.
    - `y_test (ArrayLike)`: Test target labels.
    - `fold (int)`: Fold identifier.

    #### Returns
    - `dict`: A dictionary containing:
        - `'fold' (int)`: The fold identifier.
        - `'train_size' (int)`: Number of samples in the training set.
        - `'test_size' (int)`: Number of samples in the test set.
        - `'accuracy' (float)`: Accuracy score for the current fold.
    """
    # TODO: Implement the evaluation of a single fold.
    pass


def run_kfold_evaluation(X: ArrayLike, y: ArrayLike, k=5, random_state=2025) -> ArrayLike:
    """
    Performs k-fold cross-validation on the given dataset using a custom decision tree evaluator.

    The function splits the dataset into `k` folds, trains and tests a decision tree model on each fold using the `evaluate_fold` function, and aggregates the results (e.g., accuracy) into a single DataFrame.

    #### Parameters
    - `X (ArrayLike)`: Feature dataset containing independent variables.
    - `y (ArrayLike)`: Target variable corresponding to `X`.
    - `k (int, default=5)`: Number of folds to use for cross-validation.
    - `random_state (int, default=2025)`: Random seed for reproducibility of fold splits.

    #### Returns
    - `ArrayLike`: A DataFrame summarizing the evaluation results for each fold,
      containing the following columns:
        - `'fold' (int)`: The fold identifier.
        - `'train_size' (int)`: Number of training samples in that fold.
        - `'test_size' (int)`: Number of test samples in that fold.
        - `'accuracy' (float)`: Accuracy score achieved on the test set.
    """
    # TODO: Implement the k-fold cross-validation process.
    pass

#### **(01.2.2)** Interpretation der Ergebnisse

Die folgende Zelle führt die zuvor implementierte k-Fold-Evaluation aus und gibt eine tabellarische Übersicht der Ergebnisse aus.

**_An der nächsten Code Zelle muss nichts verändert werden._**

In [ ]:
results_df = run_kfold_evaluation(X, y, k=5)
print(results_df)

Was fällt eucht bei den Ergebnissen auf? Beschreibt kurz, welche Muster oder Auffälligkeiten ihr erkennen könnt. Geht dabei insbesondere darauf ein, was die Resultate über die Generalisierungsfähigkeit des Modells aussagen. Nehmt dabei Bezug auf das Training aus Teilaufgabe 0.1.1.3. Was sagen die Ergebnisse über die Implementierung euerer `DecisionTree` Klasse aus?

> _Interpretation der Ergebnisse:_

Ist die Accuracy hier eine geeignete Metrik? Welche Alternativen gäbe es?

> _Diskussion über Metriken:_

### 3 | ILP
_Für insgesamt 16 Punkte_

Eine Hexe, hat ihr ganzes Leben damit verbracht, Rituale zu dokumentieren. In ihrem digitalisierten Ritual-Journal sind über siebzig Rituale festgehalten, von denen einige gefährliche Monster beschworen, während andere harmlos verliefen. Jedes Ritual ist genau beschrieben: Die verwendeten Zutaten, der Ort, die Mondphase und Besonderheiten wie die Frage, ob die Beschwörungsformeln rückwärts gesprochen wurden oder das Ritual um Mitternacht stattfand.

Eure Aufgabe besteht darin, ein Python-Programm zu entwickeln, das mithilfe von `janus_swi` die Prolog-Fakten aus der Datei `rituals.pl` abfragt und analysiert, welche Bedingungen besonders stark mit gefährlichen Beschwörungen in Zusammenhang stehen**.
Hierzu soll der aus der Vorlesung bekannte _FoilGain_ berechnet werden.

Die FoilGain-Formel lautet:

$$
\text{FoilGain}(L, R) = t \cdot \Big(\log_2\frac{p_1}{p_1+n_1} - \log_2\frac{p_0}{p_0+n_0}\Big)
$$

mit

- $L$ als neuem Literal, das zu ($R$) hinzugefügt wurde, um die neue Regel ($R'$) zu erhalten,
- $t$ als die Anzahl der positiven Beispiele der Regel ($R$), die auch noch von ($R'$) abgedeckt sind,
- $p_0$ als Anzahl der positiven Beispiele, die **vor** Hinzufügen von ($L$) abgedeckt werden,
- $n_0$ als Anzahl der negativen Beispiele, die **vor** Hinzufügen von ($L$) abgedeckt werden,
- $p_1$ als Anzahl der positiven Beispiele, die **nach** Hinzufügen von ($L$) abgedeckt werden,
- $n_1$ als Anzahl der negativen Beispiele, die **nach** Hinzufügen von ($L$) abgedeckt werden.

Die FoilGain-Formel bewertet also, _wie stark eine neue Bedingung (Literal)_ die Unterscheidung zwischen gefährlichen und harmlosen Ritualen verbessert.
Ein hoher FoilGain-Wert bedeutet, dass das Literal den Anteil gefährlicher Rituale unter den abgedeckten Beispielen deutlich erhöht – und damit ein wichtiger Hinweis auf gefährliche Beschwörungsbedingungen ist.

<br>

**Vorgehensweise und Anforderungen für die Implementierung:**

* Nutzt `janus_swi`, um die Fakten in `rituals.pl` abzufragen.

* Implementiert eine Funktion `calculate_foil_gain(p0, n0, p1, n1)`, die den FoilGain nach der angegebenen Formel berechnet.

* Erstellt eine Hilfsfunktion, die für eine beliebige Bedingung (Literal) zählt, wie viele Rituale gefährlich und wie viele sicher sind.

* Die möglichen Bedingungen (Kandidaten-Literale) dürfen _nicht fest im Code hinterlegt_ sein. Stattdessen sollen sie  _automatisch_ aus den vorhandenen Fakten in `rituals.pl` generiert werden.
  Dadurch bleibt das Programm erweiterbar, falls neue `ingredients`, `moon_phases` oder `locations` hinzukommen.

- Die Prolog-Abfragen können z. B. so aussehen:
  `"summoned(R, monster)."` für gefährliche Rituale oder `"ingredient(R, nightshade)."` als zusätzliche Bedingung.
  Somit gelten Rituale mit `"summoned(R, monster)."` als _positive Beispiele_, und solche mit `"summoned(R, nothing)."` als _negative Beispiele_.

- Implementiert anschließend eine Routine, die _alle möglichen Kandidaten-Literale automatisch erzeugt_ und für jedes Literal:
  1. ($p_0$, $n_0$, $p_1$, $n_1$) ermittelt,
  2. den FoilGain berechnet,
  3. das Ergebnis (Literal + FoilGain-Wert) in einer Liste oder Tabelle speichert.

- Sortiert alle berechneten Bedingungen nach ihrem FoilGain-Wert.

- Gebt am Ende die _fünf gefährlichsten Bedingungen_ (mit dem höchsten FoilGain) aus.

- Testet eure Implementierung exemplarisch an einzelnen Bedingungen, z. B. bestimmten Zutaten, Mondphasen, Orten oder Spezialattributen (`spoken_backwards`, `performed_at_midnight`).

In [ ]:
# TODO: Aufgabe 3

>